In [1]:
import pandas as pd
import numpy as np
# 원래 데이터 병합
data1 = {
    '산업 분류': ['농림수산업', '광업', '제조업', '건설업', '도소매업', '숙박음식', '전기가스수도', '운수운편', '금융·보험', '정보통신', '부동산업', '전문 기술 서비스', '교육', '보건·복지', '기타 서비스'],
    '2012': [6108, 454, 259378, 136569, 392782, 158606, 16665, 108440, 32833, 45030, 40075, 41424, 70296, 245665, 147885],
    '2016': [7876, 403, 223442, 136561, 452111, 175940, 18147, 116686, 28291, 36329, 46937, 47333, 74691, 304039, 143744]
}

data2 = {
    '산업 분류': ['농림수산업', '광업', '제조업', '건설업', '도소매업', '숙박음식', '전기가스수도', '운수운편', '금융·보험', '정보통신', '부동산업', '전문 기술 서비스', '공공행정', '교육', '보건·복지', '기타 서비스'],
    '2017': [90717, 877, 303872, 196066, 455205, 154700, 25960, 168087, 61281, 58907, 42171, 206825, 88237, 95705, 340086, 170668],
    '2018': [83817, 727, 304835, 198321, 432265, 154531, 25279, 164651, 56888, 59795, 42305, 209764, 85073, 96805, 358648, 171002],
    '2019': [78045, 824, 307491, 201812, 433938, 152298, 24716, 167006, 58036, 60586, 42792, 214038, 86191, 96098, 362879, 172686],
    '2020': [68272, 925, 310328, 204294, 438314, 151109, 24286, 170120, 59525, 61716, 43547, 218230, 87790, 95860, 369179, 171851],
    '2021': [76064, 1046, 310899, 204722, 441587, 148767, 25746, 173122, 61145, 63044, 45705, 224687, 89706, 96722, 375822, 174389],
    '2022': [None, 1433, None, None, 435654, 152108, None, 169085, 57260, 62774, None, None, 87137, 99311, 375743, None]
}

# DataFrame으로 변환
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# 병합
df = pd.merge(df1, df2, on='산업 분류', how='outer')

# 산업 분류 → 대분류 매핑
industry_to_group = {
    '농림수산업': '농업, 임업 및 어업 (A)',
    '광업': '광·제조업 (B,C)',
    '제조업': '광·제조업 (B,C)',
    '건설업': '건설업 (F)',
    '도소매업': '도소매·숙박음식업 (G,I)',
    '숙박음식': '도소매·숙박음식업 (G,I)',
    '전기가스수도': '전기·운수·통신·금융 (D,H,J,K)',
    '운수운편': '전기·운수·통신·금융 (D,H,J,K)',
    '금융·보험': '전기·운수·통신·금융 (D,H,J,K)',
    '정보통신': '전기·운수·통신·금융 (D,H,J,K)',
    '부동산업': '사업·개인·공공서비스 및 기타 (E,L~U)',
    '전문 기술 서비스': '사업·개인·공공서비스 및 기타 (E,L~U)',
    '공공행정': '사업·개인·공공서비스 및 기타 (E,L~U)',
    '교육': '사업·개인·공공서비스 및 기타 (E,L~U)',
    '보건·복지': '사업·개인·공공서비스 및 기타 (E,L~U)',
    '기타 서비스': '사업·개인·공공서비스 및 기타 (E,L~U)',
}

# 대분류 컬럼 추가
df['대분류'] = df['산업 분류'].map(industry_to_group)

# 연도형 컬럼만 자동으로 추출
years = [col for col in df.columns if col.isdigit()]

# 모든 연도(2012~2022) 생성
all_years = [str(year) for year in range(2012, 2023)]

# 결측 연도(2013, 2014, 2015) 추가
for year in ['2013', '2014', '2015']:
    if year not in df.columns:
        df[year] = np.nan
        
        
# 연도 컬럼 정렬
years = sorted([col for col in df.columns if col.isdigit()])

# 산업별로 선형 보간 수행
for industry in df['산업 분류']:
    industry_data = df[df['산업 분류'] == industry][years].T
    if industry_data.isna().any().any():
        industry_data.index = pd.to_numeric(industry_data.index)
        industry_data = industry_data.interpolate(method='linear', limit_direction='both')
        for year in years:
            df.loc[df['산업 분류'] == industry, year] = industry_data.loc[int(year), industry_data.columns[0]]

# 그룹화 및 합산
df_grouped = df.groupby('대분류')[years].sum().reset_index()

# 결과 출력
print(df_grouped)

                        대분류      2012       2013      2014       2015  \
0                   건설업 (F)  136569.0  136567.00  136565.0  136563.00   
1               광·제조업 (B,C)  259832.0  250835.25  241838.5  232841.75   
2           농업, 임업 및 어업 (A)    6108.0    6550.00    6992.0    7434.00   
3           도소매·숙박음식업 (G,I)  551388.0  570553.75  589719.5  608885.25   
4  사업·개인·공공서비스 및 기타 (E,L~U)  633582.0  651431.75  669281.5  687131.25   
5     전기·운수·통신·금융 (D,H,J,K)  202968.0  202089.25  201210.5  200331.75   

       2016    2017    2018    2019    2020     2021       2022  
0  136561.0  196066  198321  201812  204294   204722   204722.0  
1  223845.0  304749  305562  308315  311253   311945   312332.0  
2    7876.0   90717   83817   78045   68272    76064    76064.0  
3  628051.0  609905  586796  586236  589423   590354   587762.0  
4  704981.0  943692  963597  974684  986457  1007031  1006972.0  
5  199453.0  314235  306613  310344  315647   323057   314865.0  


In [14]:
import pandas as pd
import numpy as np
import random

# 기존 데이터프레임(df_grouped) 사용
# df_grouped는 대분류별로 연도(2012~2022) 데이터가 집계된 상태

# 연도 컬럼
years = [str(year) for year in range(2012, 2023)]

# 반기별 컬럼 이름 생성 (2012_H1, 2012_H2, ..., 2022_H2)
half_years = []
for year in range(2012, 2023):
    half_years.extend([f"{year}_H1", f"{year}_H2"])

# 새로운 데이터프레임 생성
df_half_yearly = pd.DataFrame(index=df_grouped['대분류'], columns=half_years)

# 대분류별로 처리
for group in df_grouped['대분류']:
    # 연도별 데이터 가져오기
    yearly_data = df_grouped[df_grouped['대분류'] == group][years].values.flatten()
    
    # 연도 데이터를 반기 단위로 확장 (랜덤 비율 적용)
    half_yearly_values = []
    for yearly_value in yearly_data:
        if not np.isnan(yearly_value):
            # 상반기 비율을 0.45~0.55 사이에서 랜덤 생성
            h1_ratio = random.uniform(0.45, 0.55)
            h2_ratio = 1 - h1_ratio
            # 상반기와 하반기 값 계산
            half_yearly_values.append(yearly_value * h1_ratio)
            half_yearly_values.append(yearly_value * h2_ratio)
        else:
            half_yearly_values.extend([np.nan, np.nan])
    
    # 반기 데이터프레임에 할당
    df_half_yearly.loc[group] = half_yearly_values
    
    # 선형 보간 수행
    df_half_yearly.loc[group] = pd.Series(df_half_yearly.loc[group].values, index=half_years).interpolate(method='linear').values

# 데이터프레임 리셋 및 컬럼 정리
df_half_yearly = df_half_yearly.reset_index().rename(columns={'index': '대분류'})

# 소수점 없이 정수로 반올림 (변환 오류 방지 포함)
df_half_yearly.iloc[:, 1:] = (
    df_half_yearly.iloc[:, 1:]
    .apply(pd.to_numeric, errors='coerce')  # 숫자로 변환 (문자나 NaN 처리)
    .round(0)                               # 반올림
    .astype('Int64')                        # 정수형으로 변환 (결측치 허용)
)



# 대분류 순서 지정: '농업, 임업 및 어업 (A)' 첫 번째, '광·제조업 (B,C)' 두 번째, 나머지는 알파벳 순
fixed_order = ['농업, 임업 및 어업 (A)', '광·제조업 (B,C)']
other_categories = [cat for cat in df_half_yearly['대분류'] if cat not in fixed_order]
other_categories.sort()  # 나머지 대분류를 알파벳 순으로 정렬
new_order = fixed_order + other_categories

# 대분류를 새로운 순서로 재정렬
df_half_yearly['대분류'] = pd.Categorical(df_half_yearly['대분류'], categories=new_order, ordered=True)
df_half_yearly = df_half_yearly.sort_values('대분류').reset_index(drop=True)

# 행과 열 전치
df_transposed = df_half_yearly.set_index('대분류').transpose().reset_index().rename(columns={'index': 'Time'})

# CSV 파일로 저장
df_transposed.to_csv('후쿠오카_산업별_종사자수.csv', index=False, encoding='utf-8')

# 결과 확인용 출력
print("CSV 파일 '후쿠오카_산업별_종사자수.csv'가 생성되었습니다.")
print("\n데이터 미리보기:")
print(df_transposed.head())

CSV 파일 '후쿠오카_산업별_종사자수.csv'가 생성되었습니다.

데이터 미리보기:
대분류     Time 농업, 임업 및 어업 (A) 광·제조업 (B,C) 건설업 (F) 도소매·숙박음식업 (G,I)  \
0    2012_H1            2807      119306   64095          289103   
1    2012_H2            3301      140526   72474          262285   
2    2013_H1            3162      127033   64927          300198   
3    2013_H2            3388      123802   71640          270356   
4    2014_H1            3741      122926   63060          295426   

대분류 사업·개인·공공서비스 및 기타 (E,L~U) 전기·운수·통신·금융 (D,H,J,K)  
0                     322088                102116  
1                     311494                100852  
2                     333425                108464  
3                     318007                 93625  
4                     342403                 94448  


C:\Users\user\AppData\Local\Temp\ipykernel_7376\2882066664.py:41: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_half_yearly.loc[group] = pd.Series(df_half_yearly.loc[group].values, index=half_years).interpolate(method='linear').values
C:\Users\user\AppData\Local\Temp\ipykernel_7376\2882066664.py:41: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_half_yearly.loc[group] = pd.Series(df_half_yearly.loc[group].values, index=half_years).interpolate(method='linear').values
C:\Users\user\AppData\Local\Temp\ipykernel_7376\2882066664.py:41: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_half_yearly.loc[group] = pd.Series(df_ha